In [1]:
%%capture
!pip install --upgrade selenium
!apt update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# Packages & Function to Get Data

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException  # Ensure this is imported
from bs4 import BeautifulSoup
from datetime import datetime
import csv
import time
import shutil
import re
import pandas as pd
from selenium.common.exceptions import TimeoutException

In [8]:
def extract_bond_links(soup):
    bond_table = soup.find("div", {"class": "table-responsive"})
    links = []
    if bond_table:
        rows = bond_table.find_all("tr")
        for row in rows:
            cols = row.find_all("td")
            if len(cols) >= 1:
                link_element = cols[0].find("a", href=True)
                if link_element:
                    links.append(link_element["href"])
    return links

def extract_bond_data(driver, url):
    # Open the bond detail page
    driver.get(url)
    bond_data = {"URL": url}  # Start with URL for traceability

    try:
        # Handle cookie banner if it appears
        try:
            cookie_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Decline')]"))
            )
            cookie_button.click()
            print("Cookie banner handled successfully (Declined).")
        except TimeoutException:
            print("Cookie banner not found. Skipping...")

        # Wait for the data table to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[@_ngcontent-boerse-frankfurt-c151]"))
        )

        # Define the data points and their XPaths
        data_points = {
            "time": "//div[@_ngcontent-boerse-frankfurt-c151]//td[contains(@class, 'widget-table-cell text-right') and text()[contains(., ':')]]",
            "bid": "//div[@_ngcontent-boerse-frankfurt-c151]//td[contains(@class, 'widget-table-cell askBidLimit') and not(contains(@class, 'text-right'))]",
            "ask": "//div[@_ngcontent-boerse-frankfurt-c151]//td[contains(@class, 'widget-table-cell askBidLimit text-right')]",
            "bid_for_x_nominal": "//div[@_ngcontent-boerse-frankfurt-c151]//td[contains(@class, 'widget-table-cell') and text()[contains(., 'für')] and not(contains(@class, 'text-right'))]",
            "ask_for_x_nominal": "//div[@_ngcontent-boerse-frankfurt-c151]//td[contains(@class, 'widget-table-cell text-right') and text()[contains(., 'für')] and not(contains(@class, 'border-bottom'))]",
            "difference_to_day_before": "//div[@_ngcontent-boerse-frankfurt-c151]//td[contains(@class, 'widget-table-cell border-bottom text-right') and (contains(@class, 'text-color-green') or contains(@class, 'text-color-red'))]",
            "spread_absolute_relative": "//div[@_ngcontent-boerse-frankfurt-c151]//td[contains(@class, 'widget-table-cell no-border text-right')]"
        }

        # Loop through each data point and extract the text
        for key, xpath in data_points.items():
            try:
                # Use find_elements to handle multiple matches and select the first one
                elements = driver.find_elements(By.XPATH, xpath)
                if elements:
                    text = elements[0].text.strip()  # Get the first element's text
                    if key in ["bid_for_x_nominal", "ask_for_x_nominal"]:
                        match = re.search(r'\d+', text)  # Extract only the numeric value
                        bond_data[key] = int(match.group()) if match else 0  # Default to 0 if no number is found
                    else:
                        bond_data[key] = text  # Save the full text for other fields
                else:
                    bond_data[key] = "N/A"  # Handle missing elements gracefully

            except Exception as inner_e:
                bond_data[key] = "N/A"  # Handle missing data gracefully
                print(f"Error extracting data for {key}: {inner_e}")

    except TimeoutException:
        print(f"Timeout: Failed to load data on page {url}")
    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")

    return bond_data

# Green bond data

In [6]:
# Check if Chromedriver is available on system path
shutil.which("chromedriver")

# Set up Chrome options to run in headless mode
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Initialize the Selenium WebDriver with the options
driver = webdriver.Chrome(options=chrome_options)  # Or use webdriver.Firefox(), etc.

# URL to scrape
url = "https://www.boerse-frankfurt.de/anleihen/most-traded"

# Script principal
try:
    # start driver
    driver.get(url)
    wait = WebDriverWait(driver, 5)

    try:
        cookie_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Decline')]"))
        )
        cookie_button.click()
        print("Cookie banner handled successfully (Declined).")
    except TimeoutException:
        print("Cookie banner not found. Skipping...")

    # 100 button
    hundred_button = wait.until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'page-bar-type-button btn btn-lg ng-star-inserted') and text()='100']"))
    )
    hundred_button.click()
    time.sleep(5)

    # Variable to store bond links
    all_bond_links = []
    # Get total number of pages
    page_buttons = driver.find_elements(
        By.XPATH,
        "//button[contains(@class, 'page-bar-type-button page-bar-type-button-width-auto btn btn-lg ng-star-inserted') and not(@disabled)]",
    )
    page_limit = 10 # Limit to 1000 bonds for now
    total_pages = min(
        int(page_buttons[-1].text.strip()), page_limit or float("inf")
    )
    print(f"Total pages varaible: {total_pages}")
    print(f"Total pages shown on website: {page_buttons[-1].text.strip()}")

    for page in range(1, total_pages + 1):
        try:
            if page != 1:
                page_button = wait.until(
                    EC.element_to_be_clickable((By.XPATH, f"//button[contains(@class, 'page-bar-type-button page-bar-type-button-width-auto btn btn-lg ng-star-inserted') and text()='{page}']"))
                )
                page_button.click()
                time.sleep(5)

            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
            bond_links = extract_bond_links(soup)
            all_bond_links.extend(bond_links)

        except Exception as e:
            print(f"Error at the page {page}: {e}")
            break

    # Collecter les static data pour chaque obligation
    detailed_bonds_data = []
    print(f"preview of bond links: {all_bond_links}")

    # getting bond data now
    # Base URL
    base_url = "https://www.boerse-frankfurt.de"

    # # Iterate through each bond link
    # for link in all_bonds_links:  # `all_bonds_links` should already be collected earlier
    #     full_url = base_url + link
    #     print(f"Processing bond URL: {full_url}")
    #     bond_data = extract_bond_data(driver, full_url)
    #     detailed_bonds_data.append(bond_data)

except Exception as e:
    print(f"Critical error: {e}")
    # add screenshot
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    screenshot_path = f"error_screenshot_{timestamp}.png"
    driver.save_screenshot(screenshot_path)

# Export the DataFrame to a CSV file even if errors occur
finally:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    # # Create a DataFrame from the data
    # df = pd.DataFrame(detailed_bonds_data)
    # # Save to CSV
    # csv_file = f"bond_spread_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    # df.to_csv(csv_file, index=False)
    # print(f"Data saved to {csv_file}")

    # Save bond links as a backup
    df_links = pd.DataFrame(all_bond_links, columns=["Bond Links"])
    csv_links_file = f"bond_links_backup_{timestamp}.csv"
    df_links.to_csv(csv_links_file, index=False)
    print(f"Bond links backup saved to {csv_links_file}")

    # Close the WebDriver
    driver.quit()

Cookie banner not found. Skipping...
Total pages varaible: 10
Total pages shown on website: 355
preview of bond links: ['/anleihe/xs2893176862-abn-amro-bank-n-v-6-375', '/anleihe/us900123cg37-tuerkei-republik-6-625-14-45', '/anleihe/gb00bpcjd880-grossbritannien-und-nord-irland-vereinigtes-koenigreich-3-5-23-25', '/anleihe/us91282cbs98-united-states-of-america-1-25-21-28', '/anleihe/xs2502885499-dz-bank-ag-deutsche-zentral-genossenschaftsbank-frankfurt-am-main-3-4-22-26', '/anleihe/de000bu22072-deutschland-bundesrepublik-2-24-26', '/anleihe/fi4000550249-finnland-republik-3-23-33', '/anleihe/xs2832873355-fraport-ag-frankfurt-airport-services-worldwide-4-25-24-32', '/anleihe/es0000012j15-spanien-koenigreich-0-000-21-27', '/anleihe/de000bu2z049-deutschland-bundesrepublik-2-5-25-35', '/anleihe/de0001102382-deutschland-bundesrepublik-1-15-25', '/anleihe/de000bu22023-deutschland-bundesrepublik-3-1-23-25', '/anleihe/eu000a2scad0-european-financial-stability-facility-efsf-1-5-22-25', '/anleihe/

In [9]:
from multiprocessing import Pool

# declare but don't define the driver yet
driver = None
base_url = "https://www.boerse-frankfurt.de"

def create_driver():
    global driver # let this function set the driver globally
    # Check if Chromedriver is available on system path
    shutil.which("chromedriver")
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    # Initialize the Selenium WebDriver with the options
    driver = webdriver.Chrome(options=chrome_options)  # Or use webdriver.Firefox(), etc.

    # don't return anything

urls = all_bond_links # your list of urls

# number of processes to use
# can be more than the number of CPUs
num_concurrent = 10

def scrape(link):
    assert driver is not None, "Driver not set up inside process"
    # Iterate through each bond link
    full_url = base_url + link
    print(f"Processing bond URL: {full_url}")
    bond_data = extract_bond_data(driver, full_url)
    return bond_data

with Pool(num_concurrent, initializer=create_driver) as p:
    detailed_bonds_data = p.map(scrape, urls)

df = pd.DataFrame(detailed_bonds_data)
# Save to CSV
csv_file = f"bond_spread_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df.to_csv(csv_file, index=False)
print(f"Data saved to {csv_file}")

Processing bond URL: https://www.boerse-frankfurt.de/anleihe/xs2893176862-abn-amro-bank-n-v-6-375
Processing bond URL: https://www.boerse-frankfurt.de/anleihe/xs2630112014-bayer-ag-4-25-23-29Processing bond URL: https://www.boerse-frankfurt.de/anleihe/us91282cly56-united-states-of-america-4-25-24-26

Processing bond URL: https://www.boerse-frankfurt.de/anleihe/de000bu22007-deutschland-bundesrepublik-2-5-23-25
Processing bond URL: https://www.boerse-frankfurt.de/anleihe/at0000a0dxc2-oesterreich-republik-4-85-09-26
Processing bond URL: https://www.boerse-frankfurt.de/anleihe/xs2441552192-the-goldman-sachs-group-inc-1-25-22-29
Processing bond URL: https://www.boerse-frankfurt.de/anleihe/xs2630111719-bayer-ag-4-625-23-33Processing bond URL: https://www.boerse-frankfurt.de/anleihe/us893830af64-transocean-inc-7-5-01-31

Processing bond URL: https://www.boerse-frankfurt.de/anleihe/gr0133009226-griechenland-republik-4-3-12-31Processing bond URL: https://www.boerse-frankfurt.de/anleihe/es000001